<img src="images/network_prof.png" width="150" alt="Prof looking at a electronic brain" style="float: left; margin-right: 15px; margin-bottom: 10px;"> Dans cette partie, nous allons créer de vrais agents pour répondre aux questions des élèves sur le contenu du cours réseau. 

Avec le professeur de philosophie, nous avons vu les interrogrations de base d'un LLM, avec des appels de bas niveau.
Cela demandait pas mal de code, et surtout les interrogrations étaient séquentielles. Quand nous avons demander à plusieurs LLM de plancer sur le devoir, il a fallu attendre la réponse
de la première pour lancer la seconde, alors qu'elles auraient faire ce travail en même temps.

Voici la liste des modules dont nous aurons besoin dans cette partie

In [41]:
from dotenv import load_dotenv
import os
from pypdf import PdfReader

from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
import gradio
import asyncio


## Retrouver le contenu du livre "Programmer l'Internet des Objets"

Dans un premier temps, nous allons convertir en texte les premières pages du livre.

In [9]:
book = PdfReader("./PLIDO_BOOK_en.pdf")
book_content = ""
for page in book.pages:
    text = page.extract_text()
    book_content += text

## Donner l'information au LLM

<img src="images/agent.png" width="150" alt="Prof looking at a electronic brain" style="float: left; margin-right: 15px; margin-bottom: 10px;">Une fois les clés API chargées pour plusieurs serveurs (on utilisera par défaut celui de l'Université de Rennes, mais l'utilisation de Gemini est aussi possible). On donne l'URI et le Token pour le service, puis dans un deuxième temps, on précise le modèle LLM utilisé. Si on utilisait par defaut OpenAI, ces lignes seraient inutiles. Lors de l'appel d'```Agent``` , dans le cas d'OpenAI, le nom du modèle est directement indiqué dans une chaîne de caractères. 

A la création de l'Agent, lui donne un identifiant pour les traces, puis les instructions qui vont lui indiquer son rôle, les limites des réponses et les actions qu'il devra faire dans certain cas. Ces instructions contiennent également l'integralité du livre en ASCII. A noter que l'on demande au LLM de ne pas chercher à répondre en détail si la réponse ne se trouve pas dans le livre.

In [34]:
load_dotenv(override=True)

rennes_api_key = os.getenv("RENNES_API_KEY")
if not rennes_api_key:
    print("RENNES_API_KEY is missing")
    exit(1)

google_api_key = os.getenv("GOOGLE_API_KEY")
if not rennes_api_key:
    print("GOOLE_API_KEY is missing")
    exit(1)
    
RENNES_BASE_URL = "https://ragarenn.eskemm-numerique.fr/sso/ch@t/api"
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

rennes_client = AsyncOpenAI(base_url=RENNES_BASE_URL, api_key=rennes_api_key)
rennes_model  = OpenAIChatCompletionsModel(model="mistralai/Mistral-Small-3.1-24B-Instruct-2503", openai_client=rennes_client)

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model  = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

instructions = f"""Voici le contenu d'un livre sur l'Internet des Objets

{book_content}

Ta responsabilité est de représenter l'auteur (Laurent Toutain) pour des interactions avec les élèves.
Les réponses doivent être professionnelles, claires, et doivent donner envie aux étudiants de s'engager
dans le cours, voire de choisir cette formation.
Si tu ne connais pas la réponse aux questions, répond Non."""

book_agent = Agent(name="PLIDO Book Agent", instructions=instructions, model=rennes_model)

Un agent se lance en utilisant la coroutine Python ```run``` du module ```Runner``` importé du module ```agents``` d'OpenAI.  L'utilisation du mot-clé ```await``` est indispensable. Ici, la différence avec une fonction est minime, vu que l'on ne lance qu'une coroutine et que l'on attend sa fin pour passer à l'instruction suivante. 

Vous pouvez relancer la cellule suivante plusieurs fois en changeant la question.

In [33]:
result = await Runner.run(book_agent, "Est ce que le livre est un bon livre à lire à la plage?")
display(Mardown(result.final_output))

Le livre "Programming the Internet of Things" de Laurent Toutain ne contient pas de recette pour préparer une omelette. Ce livre est axé sur la programmation et l'architecture des systèmes de l'Internet des objets (IoT). Il couvre des sujets tels que les protocoles de communication, les architectures réseau, la représentation des données, et les implémentations pratiques avec des exemples de code en Python.

Si vous cherchez une recette pour préparer une omelette, vous pouvez consulter des livres de cuisine ou des sites web spécialisés en recettes culinaires. Voici une recette simple pour préparer une omelette :

### Recette de l'Omelette

#### Ingrédients :
- 2 à 3 œufs
- 2 cuillères à soupe de lait (facultatif)
- Sel et poivre
- 1 cuillère à soupe de beurre

#### Instructions :
1. **Préparation des œufs** : Cassez les œufs dans un bol. Ajoutez le lait (si utilisé), du sel et du poivre. Battez les œufs avec une fourchette jusqu'à obtenir un mélange homogène.

2. **Chauffage de la poêl

In [42]:
async def chat_async(message, history):
    """Fonction async pour l'agent"""
    try:
        result = await Runner.run(book_agent, message)
        return result.final_output
    except Exception as e:
        return f"Erreur: {e}"

def chat_fn(message, history):
    """Fonction sync pour ChatInterface"""
    return asyncio.run(chat_async(message, history)) 

gradio.ChatInterface(chat_fn, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
